In [75]:
from lazyme import per_section
import nltk 
from collections import Counter
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from lazyme import per_section
import nltk 
from collections import Counter

training_sentences = [[tuple(token.split('\t')) for token in sent] for sent in per_section(open('C:\\Users\\HP\\Desktop\\TEST\\Training\\L1\\Train_L1.txt'))]
test_sentences = [[tuple(token.split('\t')) for token in sent] for sent in per_section(open('C:\\Users\\HP\\Desktop\\TEST\\Testing\\L1\\Test_L1.txt'))]


In [76]:
print (training_sentences[0])

[('المادة', 'N'), ('الثالثة', 'A'), ('و', 'A'), ('العشرون', 'A'), (':', 'PUNC'), ('تحمي', 'V'), ('الدولة', 'N'), ('عقيدة', 'N'), ('الإسلام', 'N'), ('،', 'PUNC'), ('و', 'RP'), ('تطبق', 'V'), ('شريعته', 'N'), ('،', 'PUNC'), ('و', 'RP'), ('تأمر', 'V'), ('ب', 'RP'), ('المعروف', 'A'), ('و', 'RP'), ('تنهى', 'V'), ('عن', 'RP'), ('المنكر', 'A'), ('،', 'PUNC'), ('و', 'RP'), ('تقوم', 'V'), ('ب', 'RP'), ('واجب', 'A'), ('الدعوة', 'N'), ('إلى', 'RP'), ('الله', 'N'), ('.', 'PUNC')]


In [86]:


def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        
        
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'prefix-4': sentence[index][:4],
        'prefix-5': sentence[index][:5],
      #  'prefix-6': sentence[index][:6],
        #'prefix-7': sentence[index][:7],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
       # 'suffix-4': sentence[index][-4:],
        #'suffix-5': sentence[index][-5:],
        #'suffix-6': sentence[index][-6:],
        #'suffix-7': sentence[index][-7:],
      #  '+1:postag': postag1,
      #  '+1:postag[:2]': postag1[:2],
        'prev_word1': '' if index == 0 else sentence[index - 1],
        
        'next_word1': '' if index == len(sentence) - 1 else sentence[index + 1],
    
      #  'is_numeric': sentence[index].isdigit(),
      #  '+1:postag': postag1,
      #  '+1:postag[:2]': postag1[:2],
    }
       


In [87]:
import pprint 

pprint.pprint(features(training_sentences[0], 2))

{'is_first': False,
 'is_last': False,
 'next_word1': ('العشرون', 'A'),
 'prefix-1': 'و',
 'prefix-2': ('و', 'A'),
 'prefix-3': ('و', 'A'),
 'prefix-4': ('و', 'A'),
 'prefix-5': ('و', 'A'),
 'prev_word1': ('الثالثة', 'A'),
 'suffix-1': 'A',
 'suffix-2': ('و', 'A'),
 'suffix-3': ('و', 'A'),
 'word': ('و', 'A')}


In [88]:
from nltk.tag.util import untag
 
# Split the dataset for training and testing

 
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        X.append([features(untag(tagged), index) for index in range(len(tagged))])
        y.append([tag for _, tag in tagged])
 
    return X, y
 
X_train, y_train = transform_to_dataset(training_sentences)
X_test, y_test = transform_to_dataset(test_sentences)

print(len(X_train))     
print(len(X_test))         
#print(X_train[0])
#print(y_train[0])


183
37


In [89]:
import time
import eli5
start_time = time.time()

from sklearn_crfsuite import CRF
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
 
crf = sklearn_crfsuite.CRF(algorithm= 'lbfgs',

    c1=0.01,
    c2=0.01,
    max_iterations=1000,
    all_possible_transitions=True)
crf.fit(X_train, y_train)

print("--- %s seconds ---" % (time.time() - start_time))
eli5.show_weights(crf, top=30)

--- 10.80099105834961 seconds ---


C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,A,ABBRPEV,D,DIGIT,FOREIGN,I,N,P,PUNC,RP,SYMB,V
A,0.417,-1.394,0.22,-1.223,-0.78,-0.604,1.207,-0.169,0.391,0.348,-0.585,0.0
ABBRPEV,-0.081,1.364,0.0,1.125,0.0,0.0,-0.427,-0.389,1.2,-0.717,0.479,-0.492
D,-0.829,0.0,0.0,-0.002,0.0,0.0,-0.958,-1.018,-0.042,0.111,0.0,0.568
DIGIT,0.0,2.252,0.0,0.054,0.0,0.0,1.51,-0.581,0.775,0.0,1.421,-1.798
FOREIGN,-0.244,0.0,0.0,0.206,3.732,0.0,-0.424,0.0,0.032,-0.006,0.0,0.0
I,-0.517,0.0,0.0,0.0,0.0,0.0,-0.044,-0.092,-0.497,-0.96,0.0,0.752
N,1.488,0.328,-1.517,0.776,0.808,0.085,2.442,-0.218,1.182,0.999,-0.024,0.567
P,-1.098,-0.007,-1.583,-0.468,-0.65,-0.236,-0.668,-0.7,-0.225,-0.047,-0.506,2.421
PUNC,-0.163,0.239,-0.268,0.181,0.829,-0.119,1.228,-1.436,0.854,0.816,-0.496,0.89
RP,0.747,-0.971,0.317,-0.217,-0.338,-0.813,1.69,0.639,-0.457,0.759,-0.519,0.837


In [90]:
labels = list(crf.classes_)

labels

['N',
 'A',
 'PUNC',
 'V',
 'RP',
 'P',
 'D',
 'I',
 'DIGIT',
 'FOREIGN',
 'ABBRPEV',
 'SYMB']

In [91]:
crf.get_params()

{'algorithm': 'lbfgs',
 'all_possible_states': None,
 'all_possible_transitions': True,
 'averaging': None,
 'c': None,
 'c1': 0.01,
 'c2': 0.01,
 'calibration_candidates': None,
 'calibration_eta': None,
 'calibration_max_trials': None,
 'calibration_rate': None,
 'calibration_samples': None,
 'delta': None,
 'epsilon': None,
 'error_sensitive': None,
 'gamma': None,
 'keep_tempfiles': None,
 'linesearch': None,
 'max_iterations': 1000,
 'max_linesearch': None,
 'min_freq': None,
 'model_filename': None,
 'num_memories': None,
 'pa_type': None,
 'period': None,
 'trainer_cls': None,
 'variance': None,
 'verbose': False}

In [92]:
from sklearn_crfsuite import metrics
 
y_pred = crf.predict(X_test)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           A      0.813     0.617     0.702       261
           D      0.900     1.000     0.947         9
           I      0.000     0.000     0.000         1
           N      0.864     0.943     0.902      1001
           P      0.988     0.955     0.971        88
           V      0.900     0.859     0.879       263
     ABBRPEV      0.000     0.000     0.000         0
       DIGIT      1.000     0.857     0.923        21
     FOREIGN      1.000     1.000     1.000         2
          RP      0.991     0.982     0.986       887
        PUNC      0.986     0.986     0.986       216
        SYMB      1.000     0.286     0.444         7

    accuracy                          0.918      2756
   macro avg      0.787     0.707     0.728      2756
weighted avg      0.918     0.918     0.916      2756



C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [93]:
import time
start_time = time.time()

from sklearn_crfsuite import metrics

y_pred = crf.predict(X_test)
print("Accuracy = ",metrics.flat_accuracy_score(y_test, y_pred))
print ("Recall =", metrics.flat_recall_score(y_test, y_pred, average='weighted',  labels=labels))
print ("Precision =", metrics.flat_precision_score(y_test, y_pred, average='weighted',  labels=labels))
print ("F1 =", metrics.flat_f1_score(y_test, y_pred, average='weighted',  labels=labels))
print ("report", metrics.flat_classification_report(y_test, y_pred, labels=labels))

print("--- %s seconds ---" % (time.time() - start_time))


C:\Users\HP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy =  0.9179970972423802
Recall = 0.9179970972423802
Precision = 0.9182023142307806
F1 = 0.9155747256044874
report               precision    recall  f1-score   support

           N       0.86      0.94      0.90      1001
           A       0.81      0.62      0.70       261
        PUNC       0.99      0.99      0.99       216
           V       0.90      0.86      0.88       263
          RP       0.99      0.98      0.99       887
           P       0.99      0.95      0.97        88
           D       0.90      1.00      0.95         9
           I       0.00      0.00      0.00         1
       DIGIT       1.00      0.86      0.92        21
     FOREIGN       1.00      1.00      1.00         2
     ABBRPEV       0.00      0.00      0.00         0
        SYMB       1.00      0.29      0.44         7

    accuracy                           0.92      2756
   macro avg       0.79      0.71      0.73      2756
weighted avg       0.92      0.92      0.92      2756

--- 0.186991

In [94]:
y_pred = crf.predict(X_test)
print ("F1 =", metrics.flat_f1_score(y_test, y_pred, average='weighted',  labels=labels))

F1 = 0.9155747256044874


In [95]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           A      0.813     0.617     0.702       261
           D      0.900     1.000     0.947         9
           I      0.000     0.000     0.000         1
           N      0.864     0.943     0.902      1001
           P      0.988     0.955     0.971        88
           V      0.900     0.859     0.879       263
     ABBRPEV      0.000     0.000     0.000         0
       DIGIT      1.000     0.857     0.923        21
     FOREIGN      1.000     1.000     1.000         2
          RP      0.991     0.982     0.986       887
        PUNC      0.986     0.986     0.986       216
        SYMB      1.000     0.286     0.444         7

    accuracy                          0.918      2756
   macro avg      0.787     0.707     0.728      2756
weighted avg      0.918     0.918     0.916      2756



In [96]:


print(y_pred)

[['V', 'N', 'RP', 'N', 'N', 'N', 'N', 'RP', 'N', 'RP', 'N', 'RP', 'V', 'N', 'PUNC', 'RP', 'RP', 'N', 'N', 'RP', 'N', 'N', 'V', 'N', 'N', 'A', 'PUNC', 'RP', 'N', 'A', 'RP', 'RP', 'N', 'A', 'N', 'PUNC', 'RP', 'N', 'A', 'RP', 'N', 'RP', 'RP', 'N', 'N', 'RP', 'PUNC'], ['RP', 'RP', 'V', 'RP', 'V', 'N', 'V', 'N', 'RP', 'N', 'N', 'RP', 'P', 'V', 'PUNC', 'RP', 'N', 'A', 'RP', 'A', 'N', 'N', 'RP', 'N', 'N', 'N', 'N', 'N', 'RP', 'A', 'PUNC'], ['V', 'N', 'N', 'A', 'RP', 'N', 'RP', 'N', 'N', 'N', 'RP', 'N', 'N', 'RP', 'N', 'N', 'PUNC', 'RP', 'V', 'N', 'RP', 'N', 'RP', 'N', 'N', 'N', 'N', 'RP', 'N', 'PUNC', 'RP', 'N', 'RP', 'N', 'RP', 'N', 'N', 'RP', 'N', 'RP', 'N', 'RP', 'P', 'PUNC', 'V', 'A', 'N', 'N', 'RP', 'A', 'N', 'N', 'RP', 'N', 'RP', 'N', 'N', 'RP', 'A', 'PUNC'], ['RP', 'N', 'RP', 'P', 'PUNC', 'V', 'N', 'N', 'RP', 'A', 'RP', 'N', 'RP', 'A', 'RP', 'A', 'A', 'RP', 'N', 'N', 'RP', 'N', 'N', 'A', 'PUNC', 'RP', 'N', 'N', 'RP', 'N', 'RP', 'V', 'RP', 'N', 'RP', 'N', 'RP', 'N', 'N', 'N', 'V', 'RP',

In [97]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
FOREIGN -> FOREIGN 3.731573
N      -> N       2.441541
P      -> V       2.420976
SYMB   -> DIGIT   2.314942
V      -> N       2.303968
DIGIT  -> ABBRPEV 2.252238
RP     -> N       1.690083
DIGIT  -> N       1.509804
N      -> A       1.487649
V      -> RP      1.438373
DIGIT  -> SYMB    1.421362
ABBRPEV -> ABBRPEV 1.364284
PUNC   -> N       1.228212
A      -> N       1.207393
ABBRPEV -> PUNC    1.200352
N      -> PUNC    1.181814
ABBRPEV -> DIGIT   1.124552
N      -> RP      0.999312
PUNC   -> V       0.889728
PUNC   -> PUNC    0.854177

Top unlikely transitions:
P      -> N       -0.667860
SYMB   -> RP      -0.689711
P      -> P       -0.699733
ABBRPEV -> RP      -0.716694
A      -> FOREIGN -0.780327
RP     -> I       -0.813292
V      -> V       -0.826931
D      -> A       -0.829487
D      -> N       -0.957515
I      -> RP      -0.959553
RP     -> ABBRPEV -0.970818
D      -> P       -1.018442
P      -> A       -1.097941
A      -> DIGIT   -1.223200
A      -> AB

In [98]:
import eli5
eli5.show_weights(crf, top=10)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,A,ABBRPEV,D,DIGIT,FOREIGN,I,N,P,PUNC,RP,SYMB,V
A,0.417,-1.394,0.22,-1.223,-0.78,-0.604,1.207,-0.169,0.391,0.348,-0.585,0.0
ABBRPEV,-0.081,1.364,0.0,1.125,0.0,0.0,-0.427,-0.389,1.2,-0.717,0.479,-0.492
D,-0.829,0.0,0.0,-0.002,0.0,0.0,-0.958,-1.018,-0.042,0.111,0.0,0.568
DIGIT,0.0,2.252,0.0,0.054,0.0,0.0,1.51,-0.581,0.775,0.0,1.421,-1.798
FOREIGN,-0.244,0.0,0.0,0.206,3.732,0.0,-0.424,0.0,0.032,-0.006,0.0,0.0
I,-0.517,0.0,0.0,0.0,0.0,0.0,-0.044,-0.092,-0.497,-0.96,0.0,0.752
N,1.488,0.328,-1.517,0.776,0.808,0.085,2.442,-0.218,1.182,0.999,-0.024,0.567
P,-1.098,-0.007,-1.583,-0.468,-0.65,-0.236,-0.668,-0.7,-0.225,-0.047,-0.506,2.421
PUNC,-0.163,0.239,-0.268,0.181,0.829,-0.119,1.228,-1.436,0.854,0.816,-0.496,0.89
RP,0.747,-0.971,0.317,-0.217,-0.338,-0.813,1.69,0.639,-0.457,0.759,-0.519,0.837


In [99]:
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [100]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(100))
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-100:])

Top positive:
8.815099 RP       prev_word1:فولاته
8.603355 A        next_word1:العشرون
6.824778 V        prefix-1:ي
6.658979 RP       next_word1:يعيق
6.375845 DIGIT    prefix-1:1
6.132080 P        next_word1:متلبسا
5.933674 P        next_word1:حوله
5.897627 RP       next_word1:يعرض
5.894782 RP       next_word1:يحير
5.890299 N        prev_word1:الدكتور
5.729455 V        prev_word1:قد
5.423629 N        suffix-1:ة
5.410262 N        prev_word1:بن
5.397700 A        next_word1:الفزيائي
5.229945 P        next_word1:يلي
5.226857 N        prev_word1:خمس
5.226857 N        next_word1:ستون
5.046335 P        next_word1:قاله
4.888154 A        prefix-3:مست
4.830266 RP       next_word1:مغالاة
4.818608 RP       prev_word1:العرار
4.818608 RP       next_word1:تحف
4.761008 RP       prev_word1:الصورية
4.704077 V        suffix-2:وا
4.590181 N        prev_word1:سبعة
4.547854 N        next_word1:يكتبان
4.489827 DIGIT    prefix-1:2
4.212123 P        next_word1:هو
4.122419 P        prefix-1:ه
4.119627 A        

In [101]:
sentence ='و اركعوا مع الراكعين'.split()

def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))

[('و', 'RP'), ('اركعوا', 'V'), ('مع', 'N'), ('الراكعين', 'N')]


In [102]:
print("Number of Transition Features ")
len(crf.transition_features_)
Counter(crf.transition_features_).most_common(100)

Number of Transition Features 


[(('FOREIGN', 'FOREIGN'), 3.731573),
 (('N', 'N'), 2.441541),
 (('P', 'V'), 2.420976),
 (('SYMB', 'DIGIT'), 2.314942),
 (('V', 'N'), 2.303968),
 (('DIGIT', 'ABBRPEV'), 2.252238),
 (('RP', 'N'), 1.690083),
 (('DIGIT', 'N'), 1.509804),
 (('N', 'A'), 1.487649),
 (('V', 'RP'), 1.438373),
 (('DIGIT', 'SYMB'), 1.421362),
 (('ABBRPEV', 'ABBRPEV'), 1.364284),
 (('PUNC', 'N'), 1.228212),
 (('A', 'N'), 1.207393),
 (('ABBRPEV', 'PUNC'), 1.200352),
 (('N', 'PUNC'), 1.181814),
 (('ABBRPEV', 'DIGIT'), 1.124552),
 (('N', 'RP'), 0.999312),
 (('PUNC', 'V'), 0.889728),
 (('PUNC', 'PUNC'), 0.854177),
 (('RP', 'V'), 0.837165),
 (('PUNC', 'FOREIGN'), 0.828687),
 (('PUNC', 'RP'), 0.815602),
 (('N', 'FOREIGN'), 0.808418),
 (('N', 'DIGIT'), 0.775771),
 (('DIGIT', 'PUNC'), 0.774656),
 (('V', 'PUNC'), 0.770696),
 (('RP', 'RP'), 0.758928),
 (('I', 'V'), 0.752123),
 (('RP', 'A'), 0.746561),
 (('V', 'A'), 0.735796),
 (('SYMB', 'N'), 0.719402),
 (('RP', 'P'), 0.638559),
 (('V', 'P'), 0.632368),
 (('D', 'V'), 0.5676

In [103]:
import eli5
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [104]:
sentence= 'عاد ب خفي حنين . شروى نقير . ثالثة الأثافي . على أحر من جمر . مرور الكرام . خرط القتاد . ل الله درك . جملة و تفصيلا . بلغ السيل الزبى . أسقط في يده . من الوهلة الأولى . في خبر كان . أثلج صدورنا . حجر الزاوية . حاز قصب السبق . فتح الله عليه . دس السم في الدسم . لا حول له و لا قوة . قضت على الأخضر و اليابس . أبشر ب طول سلامة . صولات و جولات . سكت دهراً و نطق كفراً . تمخض الجبل ف ولد فأراً . الأفعال أبلغ من الأقوال . أفرغ من فؤاد أم موسى . عن بكرة أبيهم . لا تبقي و لا تذر . قرة عين . سبق السيف العذل . أحرص من حرباء .  . سد الباب . نافلة القول . صناع القرار . تفاقم الهوة . دائرة الشك . تحت عنوان . التي هي أحسن . نال استحسان . هدف ب إمضاء . يثير تساؤل . موقف مشترك . تسوية الأزمة . إنفراج أزمة . تكريس الجهود . يسود الموقف . عملية السلام . فضلا عن . خطوة رادعة . تراوح مكانها . شأنه شأن . الأمر ب رمته . في ذيل القائمة . بات من المستحيل . إنخرط في . في أعقاب . كشف أوراق . حذر من مغبة . تحت الطاولة . الوهلة الأولى . تولى رئاسة .  . عصف ذهني . المبالغ المدفوعة ل ترخيص برامج مايكروسوفت . أحدث . جنحة يرتكبها أغلب المواطنين . عارضة أزياء أولى . و سائط معلوماتية سريعة جداً . تفجير داخلي . أحدث . متجر شامل . مشغل أسطوانات . ترقيم . تجنب دفع الضرائب . رجل عصابات على ظهر دابة . انتاج غير احترافي للأفلام . مدير التسويق . تجارة المخدرات . نداء . إتجار ب البشر . تقليد السيارات . التأثير السلبي ل إنجذاب الناس ل الأخبار و ترك العمل . رغبة الحصول على المال ب لا مشقة . مؤيد ل المرشح باراك أوباما . صناعة هامشية . انتحاري تفجير . بحث في الإنترنت ب استخدام محرك بحث غوغول . كرة تتبع . مثار ل الشائعات . جهة تروج ل سلعة تجارية ب موقع على الإنترنت . رحلة برية . جراحة تجميل تعويضية ب استخدام اللدائن . ' .split()

def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))

[('عاد', 'V'), ('ب', 'RP'), ('خفي', 'N'), ('حنين', 'N'), ('.', 'PUNC'), ('شروى', 'N'), ('نقير', 'N'), ('.', 'PUNC'), ('ثالثة', 'A'), ('الأثافي', 'N'), ('.', 'PUNC'), ('على', 'RP'), ('أحر', 'N'), ('من', 'RP'), ('جمر', 'N'), ('.', 'PUNC'), ('مرور', 'N'), ('الكرام', 'N'), ('.', 'PUNC'), ('خرط', 'N'), ('القتاد', 'N'), ('.', 'PUNC'), ('ل', 'RP'), ('الله', 'N'), ('درك', 'N'), ('.', 'PUNC'), ('جملة', 'N'), ('و', 'RP'), ('تفصيلا', 'V'), ('.', 'PUNC'), ('بلغ', 'N'), ('السيل', 'N'), ('الزبى', 'N'), ('.', 'PUNC'), ('أسقط', 'N'), ('في', 'RP'), ('يده', 'N'), ('.', 'PUNC'), ('من', 'RP'), ('الوهلة', 'N'), ('الأولى', 'A'), ('.', 'PUNC'), ('في', 'RP'), ('خبر', 'N'), ('كان', 'RP'), ('.', 'PUNC'), ('أثلج', 'V'), ('صدورنا', 'N'), ('.', 'PUNC'), ('حجر', 'N'), ('الزاوية', 'N'), ('.', 'PUNC'), ('حاز', 'N'), ('قصب', 'N'), ('السبق', 'N'), ('.', 'PUNC'), ('فتح', 'N'), ('الله', 'N'), ('عليه', 'RP'), ('.', 'PUNC'), ('دس', 'N'), ('السم', 'N'), ('في', 'RP'), ('الدسم', 'N'), ('.', 'PUNC'), ('لا', 'RP'), ('حول', 'N')

In [105]:
print("Number of State Features ",len(crf.state_features_))
Counter(crf.state_features_).most_common(20)

Number of State Features  15943


[(('prev_word1:فولاته', 'RP'), 8.815099),
 (('next_word1:العشرون', 'A'), 8.603355),
 (('prefix-1:ي', 'V'), 6.824778),
 (('next_word1:يعيق', 'RP'), 6.658979),
 (('prefix-1:1', 'DIGIT'), 6.375845),
 (('next_word1:متلبسا', 'P'), 6.13208),
 (('next_word1:حوله', 'P'), 5.933674),
 (('next_word1:يعرض', 'RP'), 5.897627),
 (('next_word1:يحير', 'RP'), 5.894782),
 (('prev_word1:الدكتور', 'N'), 5.890299),
 (('prev_word1:قد', 'V'), 5.729455),
 (('suffix-1:ة', 'N'), 5.423629),
 (('prev_word1:بن', 'N'), 5.410262),
 (('next_word1:الفزيائي', 'A'), 5.3977),
 (('next_word1:يلي', 'P'), 5.229945),
 (('prev_word1:خمس', 'N'), 5.226857),
 (('next_word1:ستون', 'N'), 5.226857),
 (('next_word1:قاله', 'P'), 5.046335),
 (('prefix-3:مست', 'A'), 4.888154),
 (('next_word1:مغالاة', 'RP'), 4.830266)]

In [106]:
Counter(crf.transition_features_).most_common()[-10:]

[(('RP', 'ABBRPEV'), -0.970818),
 (('D', 'P'), -1.018442),
 (('P', 'A'), -1.097941),
 (('A', 'DIGIT'), -1.2232),
 (('A', 'ABBRPEV'), -1.394308),
 (('PUNC', 'P'), -1.436285),
 (('SYMB', 'V'), -1.465637),
 (('N', 'D'), -1.516662),
 (('P', 'D'), -1.582698),
 (('DIGIT', 'V'), -1.797912)]

In [107]:
print("Number of State Features ",len(crf.state_features_))
Counter(crf.state_features_).most_common(20)

Number of State Features  15943


[(('prev_word1:فولاته', 'RP'), 8.815099),
 (('next_word1:العشرون', 'A'), 8.603355),
 (('prefix-1:ي', 'V'), 6.824778),
 (('next_word1:يعيق', 'RP'), 6.658979),
 (('prefix-1:1', 'DIGIT'), 6.375845),
 (('next_word1:متلبسا', 'P'), 6.13208),
 (('next_word1:حوله', 'P'), 5.933674),
 (('next_word1:يعرض', 'RP'), 5.897627),
 (('next_word1:يحير', 'RP'), 5.894782),
 (('prev_word1:الدكتور', 'N'), 5.890299),
 (('prev_word1:قد', 'V'), 5.729455),
 (('suffix-1:ة', 'N'), 5.423629),
 (('prev_word1:بن', 'N'), 5.410262),
 (('next_word1:الفزيائي', 'A'), 5.3977),
 (('next_word1:يلي', 'P'), 5.229945),
 (('prev_word1:خمس', 'N'), 5.226857),
 (('next_word1:ستون', 'N'), 5.226857),
 (('next_word1:قاله', 'P'), 5.046335),
 (('prefix-3:مست', 'A'), 4.888154),
 (('next_word1:مغالاة', 'RP'), 4.830266)]

In [ ]:
Counter(crf.state_features_).most_common()[-20:]

In [ ]:
import os
print(os.getcwd())
